In [ ]:
# importa todo y define una función de ejecución de pruebas
from importlib import reload
from ayudante import ejecutar_prueba

import bloque
import cce
import ayudante
import bloquemerkle
import network
import script
import tx

### Ejercicio 1

#### Comprueba en la chuleta la estructura del mensaje de red.

#### 1.1. Parsea este mesnsaje
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 1.2. Haz que pasen [estas pruebas](/edit/sesión7/network.py).
```
network.py:PruebaRed:prueba_parsear
network.py:PruebaRed:prueba_serializar
```

In [ ]:
# Ejercicio 1.1

msj = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')

# los primeros 4 son la magia de red
magia = msj[:4]
# los siguientes 12 son el comando
comando = msj[4:16]
# los siguientes 4 son la longitud de carga
longitud_carga = msj[16:20]
# los siguientes 4 son el checksum
checksum = msj[20:24]
# el resto es la carga
carga = msj[24:]
print(comando)

b'verack\x00\x00\x00\x00\x00\x00'


In [ ]:
# Ejercicio 1.2

reload(network)
ejecutar_prueba(network.PruebaEnvelopeRed('prueba_parsear'))
ejecutar_prueba(network.PruebaEnvelopeRed('prueba_serializar'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK
.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


### Ejercicio 2

#### 2.1. Haz que pase [esta prueba](/edit/sesión7/network.py).
```
network.py:PruebaMensajeVersión:prueba_serializar
```

In [ ]:
# Ejercicio 2.1

reload(network)
ejecutar_prueba(network.PruebaMensajeVersión('prueba_serializar'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


### Ejercicio 3

#### 3.1. Haz que pase [esta prueba](/edit/sesión7/network.py).
```
network.py:PruebaObtenerMensajeCabeceras:prueba_serializar
```

#### 3.2. Haz que pase [esta prueba](/edit/sesión7/network.py).
```
network.py:MensajeCabecerasTest:prueba_parsear
```

In [ ]:
# Ejercicio 3.1

reload(network)
ejecutar_prueba(network.PruebaObtenerMensajeCabeceras('prueba_serializar'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejercicio 3.2

reload(network)
ejecutar_prueba(network.MensajeCabecerasTest('prueba_parsear'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejemplo de Handshake

from network import NodoSimple, MensajeVersión

node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True, logging=True)

version = MensajeVersión()
node.send(version.comando, version.serializar())
print(node.esperar_comandos([b'verack']))

enviando: version: 7f1101000000000000000000cfcebe5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d2068a13507a68ba4641b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
recibiendo: version: 7f1101000d04000000000000d0cebe5b00000000000000000000000000000000000000000000ffff4830fd33fefc0d0400000000000000000000000000000000000000000000000013eb2801024c8282102f5361746f7368693a302e31372e302fe8e7150001
enviando: verack: 
recibiendo: verack: 
verack: 


### Ejercicio 4

#### 4.1. Haz que pase [esta prueba](/edit/sesión7/network.py).
```
network.py:PruebaNodoSimple:prueba_handshake
```

In [ ]:
# Ejercicio 4.1

reload(network)
ejecutar_prueba(network.PruebaNodoSimple('prueba_handshake'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.088s

OK


In [ ]:
# Ejemplo de descarga de cabecera de bloque

from bloque import HASH_BLOQUE_GENESIS
from network import ObtenerMensajeCabeceras, MensajeCabeceras, NodoSimple

node = NodoSimple('btc.programmingbloquechain.com', testnet=False)
node.handshake()
hash_último_bloque = HASH_BLOQUE_GENESIS
contador = 1
for _ in range(20):
    obtenercabeceras = ObtenerMensajeCabeceras(bloque_inicial=hash_último_bloque)
    node.send(obtenercabeceras.comando, obtenercabeceras.serializar())
    envelope_cabeceras = node.esperar_comandos([b'headers'])
    mensaje_cabeceras = MensajeCabeceras.parsear(envelope_cabeceras.stream())
    for b in mensaje_cabeceras.bloques:
        if not b.comprobar_pow():
            raise RuntimeError('mala prueba de trabajo en el bloque {}'.format(contador))
        if hash_último_bloque != HASH_BLOQUE_GENESIS and b.bloque_previo != hash_último_bloque:
            raise RuntimeError('bloque discontinuo en {}'.format(contador))
        contador += 1
        hash_último_bloque = b.hash()
        if contador % 2016 == 0:
            print(b.hash().hex())

00000000693067b0e6b440bc51450b9f3850561b07f6d3c021c54fbd6abb9763
00000000f037ad09d0b05ee66b8c1da83030abaf909d2b1bf519c3c7d2cd3fdf
000000006ce8b5f16fcedde13acbc9641baa1c67734f177d770a4069c06c9de8
00000000563298de120522b5ae17da21aaae02eee2d7fcb5be65d9224dbd601c
000000009b0a4b2833b4a0aa61171ee75b8eb301ac45a18713795a72e461a946
00000000fa8a7363e8f6fdc88ec55edf264c9c7b31268c26e497a4587c750584
000000008ac55b5cd76a5c176f2457f0e9df5ff1c719d939f1022712b1ba2092
000000007f0c796631f00f542c0b402d638d3518bc208f8c9e5d29d2f169c084
00000000ffb062296c9d4eb5f87bbf905d30669d26eab6bced341bd3f1dba5fd
0000000074c108842c3ec2252bba62db4050bf0dddfee3ddaa5f847076b8822f
0000000067dc2f84a73fbf5d3c70678ce4a1496ef3a62c557bc79cbdd1d49f22
00000000dbf06f47c0624262ecb197bccf6bdaaabc2d973708ac401ac8955acc
000000009260fe30ec89ef367122f429dcc59f61735760f2b2288f2e854f04ac
00000000f9f1a700898c4e0671af6efd441eaf339ba075a5c5c7b0949473c80b
000000005107662c86452e7365f32f8ffdc70d8d87aa6f78630a79f7d77fbfe6
00000000984f962134a7291e3

### Ejercicio 5

#### 5.1. Descara los primeros 40,000 bloques para testnet y valídalos.


In [ ]:
# Ejercicio 5.1

from bloque import HASH_BLOQUE_GÉNESIS_TESTNET
from network import ObtenerMensajeCabeceras, MensajeCabeceras, NodoSimple

# conéctate a tbtc.programmingbloquechain.com
node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True)
# handshake
node.handshake()
# establece el hash del último bloque al HASH_BLOQUE_GÉNESIS_TESTNET
hash_último_bloque = HASH_BLOQUE_GÉNESIS_TESTNET
# establece el contador del bloque a 1
contador = 1
# haz un bucle hasta que tengamos 40,000 bloques
while contador < 40000:
    # crea un ObtenerMensajeCabeceras empezando por el último bloque que tengamos
    obtenercabeceras = ObtenerMensajeCabeceras(bloque_inicial=hash_último_bloque)
    # envía el comando obtenercabeceras
    node.send(obtenercabeceras.comando, obtenercabeceras.serializar())
    # espera por el comando cabeceras en la respuesta
    envelope_cabeceras = node.esperar_comandos([b'headers'])
    # parsea el envelope de red que volverá
    mensaje_cabeceras = MensajeCabeceras.parsear(envelope_cabeceras.stream())
    # haz un bucle a través de los bloques desde el mensaje de cabeceras
    for b in mensaje_cabeceras.bloques:
        # comprueba la prueba de trabajo
        if not b.comprobar_pow():
            raise RuntimeError('mala prueba de trabajo en el bloque {}'.format(contador))
        # el bloque_previo del bloque actual debería ser el último bloque
        if hash_último_bloque != HASH_BLOQUE_GÉNESIS_TESTNET and b.bloque_previo != hash_último_bloque:
            raise RuntimeError('bloque discontinuo en {}'.format(contador))
        # incrementa el contador de bloques
        contador += 1
        # establece el hash del último bloque
        hash_último_bloque = b.hash()
        # imprime la cabecera del bloque cada 2016 bloques (ajuste de dificultad)
        if contador % 2016 == 0:
            print(b.hash().hex())

00000000864b744c5025331036aa4a16e9ed1cbb362908c625272150fa059b29
000000002e9ccffc999166ccf8d72129e1b2e9c754f6c90ad2f77cab0d9fb4c7
0000000009b9f0436a9c733e2c9a9d9c8fe3475d383bdc1beb7bfa995f90be70
000000000a9c9c79f246042b9e2819822287f2be7cd6487aecf7afab6a88bed5
000000003a7002e1247b0008cba36cd46f57cd7ce56ac9d9dc5644265064df09
00000000061e01e82afff6e7aaea4eb841b78cc0eed3af11f6706b14471fa9c8
000000003911e011ae2459e44d4581ac69ba703fb26e1421529bd326c538f12d
000000000a5984d6c73396fe40de392935f5fc2a8e48eedf38034ce0a3178a60
000000000786bdc642fa54c0a791d58b732ed5676516fffaeca04492be97c243
000000001359c49f9618f3ee69afbd1b3196f1832acc47557d42256fcc6b7f48
00000000270dde98d582af35dff5aed02087dad8529dc5c808c67573d6dabaf4
00000000425c160908c215c4adf998771a2d1c472051bc58320696f3a5eb0644
0000000006a5976471986377805d4a148d8822bb7f458138c83f167d197817c9
000000000318394ea17038ef369f3cccc79b3d7dfda957af6c8cd4a471ffa814
000000000ad4f9d0b8e86871478cc849f7bc42fb108ebec50e4a795afc284926
000000000207e63e68f2a7a4c

In [ ]:
# Ejemplo de Padre Merkle

from ayudante import doble_sha256

tx_hash0 = bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')

padre_merkle = doble_sha256(tx_hash0+tx_hash1)
print(padre_merkle.hex())

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd


### Ejercicio 6

#### 6.1. Calcula el padre Merkle de estos hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```

#### 6.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py)
```
ayudante.py:PruebaAyudante:prueba_padre_merkle
```

In [ ]:
# Ejercicio 6.1

from ayudante import doble_sha256

hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'

# bytes.fromhex para obtener los hashes binarios
hash1 = bytes.fromhex(hex_hash1)
hash2 = bytes.fromhex(hex_hash2)
# doble_sha256 la combinación
padre = doble_sha256(hash1+hash2)
# hex() para ver el resultado
print(padre.hex())

7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800


In [ ]:
# Ejercicio 6.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_padre_merkle'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejemplo de nivel de Padre Merkle

from ayudante import doble_sha256, padre_merkle
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
padre_nivel = []
for i in range(0, len(hex_hashes), 2):
    padre = padre_merkle(hashes[i], hashes[i+1])
    print(padre.hex())
    padre_nivel.append(padre)

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
3ecf6115380c77e8aae56660f5634982ee897351ba906a6837d15ebc3a225df0


### Ejercicio 7

#### 7.1. Calcula el siguiente nivel de Padre Merkle dados estos hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```

#### 7.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py).
```
ayudante.py:PruebaAyudante:prueba_nivel_padre_merkle
```

In [ ]:
# Ejercicio 7.1

from ayudante import padre_merkle

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]

# bytes.fromhex para obtener todos los hashes en binario
hashes = [bytes.fromhex(h) for h in hex_hashes]
# si el número de hashes es impar, dsubirlica el último
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
# inicializa el nivel padre
padre_nivel = []
# sata de dos en dos: usa range(0, len(hashes), 2)
for i in range(0, len(hashes), 2):
    # calcula el padre_merkle de los hashes i y i+1
    padre = padre_merkle(hashes[i], hashes[i+1])
    # imprime el hex del hash
    print(padre.hex())
    # añade el padre al nivel padre
    padre_nivel.append(padre)

26906cb2caeb03626102f7606ea332784281d5d20e2b4839fbb3dbb37262dbc1
717a0d17538ff5ad2c020bab38bdcde66e63f3daef88f89095f344918d5d4f96
d6c56a5281021a587f5a1e0dd4674bff012c69d960136d96e6d72261d5b696ae


In [ ]:
# Ejercicio 7.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_nivel_padre_merkle'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejemplo de Raíz Merkle

from ayudante import doble_sha256, nivel_padre_merkle

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddcce9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]

nivel_actual = [bytes.fromhex(x) for x in hex_hashes]
while len(nivel_actual) > 1:
    nivel_actual = nivel_padre_merkle(nivel_actual)
print(nivel_actual[0].hex())

acbcab8bcc1af95d8d563b77d24c3d19b18f1486383d75a5085c4e86c86beed6


### Ejercicio 8

#### 8.1. Calcula la Raíz Merkle dados estos hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 8.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py).
```
ayudante.py:PruebaAyudante:prueba_raíz_merkle
```

In [ ]:
# Ejercicio 8.1

from ayudante import doble_sha256, nivel_padre_merkle
hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex para obtener todos los hashes en binario
hashes = [bytes.fromhex(h) for h in hex_hashes]
# inicializa el nivel actual para que sean los hashes
nivel_actual = hashes
# haz un bucle hasta que el nivel actual tenga 1 solo elemento
while len(nivel_actual) > 1:
    # haz el nivel actual el nivel padre
    nivel_actual = nivel_padre_merkle(nivel_actual)
# imprime el hex de la raíz
print(nivel_actual[0].hex())

a67772634e542799333c6c98bc903e36b652918a8d8a9e069391c55b4276c8a1


In [ ]:
# Ejercicio 8.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_raíz_merkle'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejemplo de Bloque de Raíz Merkle Example

from ayudante import doble_sha256, nivel_padre_merkle, raíz_merkle
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
nivel_actual = [bytes.fromhex(x)[::-1] for x in tx_hex_hashes]
print(raíz_merkle(nivel_actual)[::-1].hex())

654d6181e18e4ac4368383fdc5eead11bf138f9b7ac1e15334e4411b3c4797d9


### Ejercicio 9

#### 9.1. Valida la raíz merkle de este bloque en Testnet:
Hash del Bloque:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

Hashes de la transacción:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 9.2. Haz que pase [esta prueba](/edit/sesión7/bloque.py).
```
bloque.py:PruebaBloque:prueba_validar_raíz_merkle
```

In [ ]:
# Ejercicio 9.1

from ayudante import doble_sha256, raíz_merkle

want = bytes.fromhex('4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301')

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex y dale la vuelta ([::-1]) para obtener todos los hashes en binario
hashes = [bytes.fromhex(h)[::-1] for h in tx_hex_hashes]
# obtén la raíz merkle
raiz = raíz_merkle(hashes)
# mira si el reverso de la raíz es el mismo que la raíz deseada
print(raiz[::-1] == want)

True


In [ ]:
# Ejercicio 9.2

reload(bloque)
ejecutar_prueba(bloque.PruebaBloque('prueba_validar_raíz_merkle'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.002s

OK


### Ejercicio 10

#### 10.1. Valida la raíz merkle para esta bloque en Testnet vía protocolo de red:
Hash del Bloque:
```
0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900
```

In [ ]:
# Ejercicio 10.1

from bloque import HASH_BLOQUE_GÉNESIS_TESTNET, Bloque
from ayudante import read_varint
from network import TIPO_DATOS_BLOQUE, MensajeObtenerDatos, NodoSimple
from tx import Tx

bloque_hex = '0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900'
bloque_hash = bytes.fromhex(bloque_hex)

# conéctate a tbtc.programmingbloquechain.com en testnet
node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True)
# handshake
node.handshake()

# crea un MensajeObtenerDatos
obtenerdatos = MensajeObtenerDatos()
# añade el bloque que queremos
obtenerdatos.add_data(TIPO_DATOS_BLOQUE, bloque_hash)
# enviar el comando obtenerdatos
node.send(obtenerdatos.comando, obtenerdatos.serializar())

# esperar por el comando bloque en la respuesta
bloque_envelope = node.esperar_comandos([b'bloque'])
# obtén el stream desde el envelope
stream = bloque_envelope.stream()
# el mensaje del bloque es la cabecera del bloque primero, así que parsea un Bloque
b = Bloque.parsear(stream)
# comprueba la prueba de trabajo
if not b.comprobar_pow():
    raise RuntimeError('prueba de trabajo mala')
# el siguiente mensaje de bloque tiene el número de transacciones así que haz read_varint
num_txs = read_varint(stream)
# inicializa la lista de tx_hashes
tx_hashes = []
# haz un bucle ese número de veces
for _ in range(num_txs):
    # parsea la transacción
    t = Tx.parsear(stream)
    # añade el hash de la transacción al listado de tx_hashes
    tx_hashes.append(t.hash())

# establece el tx_hashes del bloque a los que hemos reunido
b.tx_hashes = tx_hashes
# valida los tx_hashes
if not b.validar_raíz_merkle():
    raise RuntimeError('mala raíz de merkle')
# imprime el hex de la raíz de merkle
print(b.raíz_merkle.hex())

627bf8053bd767ad72c6afcd2d91638311f9c7520905a634be13aa8853f7a446


In [ ]:
# Ejemplo de Árbol de Merkle

import math

total = 16

profundidad_máxima = math.ceil(math.log(total, 2))
árbol_merkle = []
for profundidad in range(profundidad_máxima + 1):
    num_cosas = math.ceil(total / 2**(profundidad_máxima - profundidad))
    nivel_hashes = [None] * num_cosas
    árbol_merkle.append(nivel_hashes)

for nivel in árbol_merkle:
    print(nivel)

[None]
[None, None]
[None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [ ]:
# Ejemplo de poblar y navegar Árbol Merkle

from bloquemerkle import ÁrbolMerkle
from ayudante import nivel_padre_merkle, padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
árbol.nodes[3] = nivel_padre_merkle(árbol.nodes[4])
árbol.nodes[2] = nivel_padre_merkle(árbol.nodes[3])
árbol.nodes[1] = nivel_padre_merkle(árbol.nodes[2])
árbol.nodes[0] = nivel_padre_merkle(árbol.nodes[1])
print(árbol)

*597c4baf.*, 
6382df3f..., 87cf8fa3..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., 8636b7a3..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., f9dbfafc..., 



In [ ]:
# Ejemplo #2 de poblar Árbol Merkle

from bloquemerkle import ÁrbolMerkle
from ayudante import padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while árbol.raiz() is None:
    if árbol.es_hoja():
        árbol.subir()
    else:
        hash_izquierdo = árbol.obtener_nodo_izquierdo()
        hash_derecho = árbol.obtener_nodo_derecho()
        if hash_izquierdo is None:
            árbol.izquierda()
        elif hash_derecho is None:
            árbol.derecha()
        else:
            árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_derecho))
            árbol.subir()
print(árbol)

597c4baf..., 
6382df3f..., 87cf8fa3..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 3df760ac..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., 8636b7a3..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., f9dbfafc..., 



In [ ]:
# Ejemplo #3 de poblar Árbol Merkle
reload(bloquemerkle)
from bloquemerkle import ÁrbolMerkle
from ayudante import padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while árbol.raiz() is None:
    if árbol.es_hoja():
        árbol.subir()
    else:
        hash_izquierdo = árbol.obtener_nodo_izquierdo()
        if hash_izquierdo is None:
            árbol.izquierda()
        elif árbol.derecha_existe():
            hash_derecho = árbol.obtener_nodo_derecho()
            if hash_derecho is None:
                árbol.derecha()
            else:
                árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_derecho))
                árbol.subir()
        else:
            árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_izquierdo))
            árbol.subir()
print(árbol)

dc87b7e3..., 
6382df3f..., 6440941e..., 
3ba6c080..., 8e894862..., 7ab01bb6..., 996be980..., 
272945ec..., 9a38d037..., 4a64abd9..., ec7c95e1..., 3b67006c..., 850683df..., d40d268b..., f542a085..., 
9745f717..., 5573c8ed..., 82a02ecb..., 507ccae5..., a7a4aec2..., bb626766..., ea6d7ac1..., 45774386..., 76880292..., b1ae7f15..., 9b74f89f..., b3a92b5b..., b5c0b915..., c9d52c5c..., c555bc5f..., 

